In [ ]:
#pip uninstall openpyxl

In [ ]:
#pip install openpyxl==3.1.0

In [ ]:
#pip install --upgrade openpyxl

In [ ]:
#pip install tabulate prettytable

In [5]:
#pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from datetime import datetime
from tabulate import tabulate

# Definir rangos para la validación de anomalías
rango_presion = (15, 20)  # Bares
rango_temperatura = (10, 50) # Grados centigrados
rango_volumen = (10, 30) # Métros cúbicos

archivo_excel = 'Datos_Contugas.xlsx'

# Cargar todas las hojas que empiezan con "CLIENTE"
clientes = pd.read_excel(archivo_excel, sheet_name=None)
resumen_datos = []

C:\Users\Olga Lucia\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Función para generar el resumen por cliente
def generar_resumen_cliente(df, nombre_hoja):
    # Convertir la columna 'Fecha' a tipo datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
    
    # 1. Granularidad
    df = df.sort_values(by='Fecha')
    granularidad = df['Fecha'].diff().min().total_seconds() / 3600 if df['Fecha'].diff().min() else None

    # 2. Valores anómalos
    num_anomalos_presion = df[(df['Presion'] < rango_presion[0]) | (df['Presion'] > rango_presion[1])].shape[0]
    num_anomalos_temperatura = df[(df['Temperatura'] < rango_temperatura[0]) | (df['Temperatura'] > rango_temperatura[1])].shape[0]
    num_anomalos_volumen = df[(df['Volumen'] < rango_volumen[0]) | (df['Volumen'] > rango_volumen[1])].shape[0]

    # 3. Registros duplicados
    registros_duplicados = df.duplicated().sum()

    # 4. Porcentaje de datos faltantes
    porcentaje_faltantes = (df.isnull().sum().sum() / df.size) * 100

    # 5. Edad promedio de los datos
    if 'Fecha' in df.columns:
        df['Edad_Datos'] = (datetime.now() - df['Fecha']).dt.days
        edad_promedio = df['Edad_Datos'].mean() if not df['Edad_Datos'].isnull().all() else None
    else:
        edad_promedio = None

    # 6. Correlación promedio
    correlacion = df[['Presion', 'Temperatura', 'Volumen']].corr().mean().mean()

    # Añadir resultados al resumen
    resumen_datos.append({
        'Cliente': nombre_hoja,
        'Granularidad (hrs)': granularidad,
        'Anomalías Presión': num_anomalos_presion,
        'Anomalías Temperatura': num_anomalos_temperatura,
        'Anomalías Volumen': num_anomalos_volumen,
        'Duplicados': registros_duplicados,
        'Faltantes (%)': round(porcentaje_faltantes, 2),
        'Edad Promedio (días)': round(edad_promedio, 2) if edad_promedio else None,
        'Correlación Promedio': round(correlacion, 2) if not pd.isna(correlacion) else None
    })

# Procesar cada hoja
for hoja, df in clientes.items():
    if hoja.startswith('CLIENTE'):
        generar_resumen_cliente(df, hoja)

# Convertir la lista a DataFrame
resumen_df = pd.DataFrame(resumen_datos)

# Exportar a un archivo Excel
nombre_archivo_excel = 'Resumen_Clientes_Optimizado.xlsx'
resumen_df.to_excel(nombre_archivo_excel, index=False)
print(f"📊 Resumen exportado exitosamente a '{nombre_archivo_excel}'")


📊 Resumen exportado exitosamente a 'Resumen_Clientes_Optimizado.xlsx'
